In [1]:
import pandas as pd
import glob

def load_salescate_data(data_folder):
    # Get the path to all files containing 'salescate' and ending with '.xlsx'
    salescate_files = glob.glob(data_folder + "/salebycategory*.xlsx")
    
    # Load all the salescate data into a list of pandas dataframes
    salescate_dfs = [pd.read_excel(file) for file in salescate_files]
    
    # Concatenate all the dataframes into one
    salescate_df = pd.concat(salescate_dfs, ignore_index=True)
    
    return salescate_df
try:
    data_folder = r'C:\Users\PC\Dropbox\Projects\data_xls'
    df = load_salescate_data(data_folder)
    print(df.head())
except Exception as e:
    data_folder = r'C:\Users\trieu.pham\Dropbox\Projects\data_xls'
    print(e)

df = load_salescate_data(data_folder)

         barcode                                itemName  divisionCode  \
0  2010101000005                         Ly đá vừa x 1ly            25   
1   260202000017           Tteokbokki Signature x 1 phần            26   
2  8934588873553          TEA+ Trà Oolong 450ml x 1 chai             7   
3  2501031190374              Nestea ly lớn (New) x 1 ly            25   
4  8934588063053  AQUAFINA Nước tinh khiết 500ml x1 Chai             7   

  divisionName  categoryCode    categoryName  subCategoryCode  \
0  FF Beverage          2501  Cold Beverages           250101   
1    FF Onsite          2602   Other Cuisine           260202   
2     Beverage           703             Tea            70301   
3  FF Beverage          2501  Cold Beverages           250102   
4     Beverage           701   Bottled Water            70101   

           subCategoryName  itemQty  grossSales  itemDiscount  netSales  \
0               Cup Of Ice      477      789000        642000    730605   
1       Korean

In [4]:
df

,barcode,itemName,divisionCode,divisionName,categoryCode,categoryName,subCategoryCode,subCategoryName,itemQty,grossSales,itemDiscount,netSales,totalCostPrice
0,2010101000005,Ly đá vừa x 1ly,25,FF Beverage,2501,Cold Beverages,250101,Cup Of Ice,477,789000,642000,730605,372060.00
1,260202000017,Tteokbokki Signature x 1 phần,26,FF Onsite,2602,Other Cuisine,260202,Korean cuisine Sub,186,5580000,0,5166708,3481242.96
2,8934588873553,TEA+ Trà Oolong 450ml x 1 chai,7,Beverage,703,Tea,70301,Sugar Tea,128,1332000,204000,1233321,1006080.00
3,2501031190374,Nestea ly lớn (New) x 1 ly,25,FF Beverage,2501,Cold Beverages,250102,Fountain-Non-carbonated,128,1536000,0,1422208,674240.00
4,8934588063053,AQUAFINA Nước tinh khiết 500ml x1 Chai,7,Beverage,701,Bottled Water,70101,Purified water,126,882000,0,816629,522018.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8733,8801728106805,YOUUS Bánh mì bơ tỏi 100g x Gói,11,Snacks,1102,Non-potato Chips,110201,Alternative Snacks,1,32000,0,29630,18375.00
8734,8809403537951,YOUUS Sugarlolo Thạch vị trái cây nhiệt đới 15...,7,Beverage,704,Juice,70401,OSC Juice,1,52000,0,48148,28900.00
8735,8850163900011,#Durex Gel bôi trơn Play Classic 50ml x chai,16,Health&Beauty,1604,Family Planning,160401,Condom/Lubricant,1,116000,0,110476,72567.00
8736,8936213150688,Bliss Kem Dừa non 80ml x hộp,5,Ice Cream,502,Premium Ice Cream,50201,Premium Ice Cream Sub,1,66000,0,61111,41000.00


In [11]:
# print all the rows which have barcode = '2010101000005

df[df['barcode'] == 8936114080084]


,barcode,itemName,divisionCode,divisionName,categoryCode,categoryName,subCategoryCode,subCategoryName,itemQty,grossSales,itemDiscount,netSales,totalCostPrice
830,8936114080084,DBMINT Kẹo cao su bạc hà 40 viên x1 1 Hủ,9,Candy,903,Gum,90301,Gum Sub,5,171000,0,158332,106920.0
3273,8936114080084,Doublemint Kẹo Cao Su Bạc Hà 40 Viên x Hủ,9,Candy,903,Gum,90301,Gum Sub,1,33000,0,30556,21384.0
4456,8936114080084,DBMINT Kẹo cao su bạc hà 40 viên x1 1 Hủ,9,Candy,903,Gum,90301,Gum Sub,4,132000,0,122224,85536.0
6478,8936114080084,Doublemint Kẹo Cao Su Bạc Hà 40 Viên x Hủ,9,Candy,903,Gum,90301,Gum Sub,2,66000,0,61112,42768.0
8114,8936114080084,Doublemint Kẹo Cao Su Bạc Hà 40 Viên x Hủ,9,Candy,903,Gum,90301,Gum Sub,3,99000,0,91668,64152.0


In [10]:
# List out all barcodes have more than one row in dataframe, print out the barcode and the number of rows for each barcode, and itemName of each barcode
df['barcode'].value_counts()[df['barcode'].value_counts() > 1]


barcode
2010101000005    5
8936114080084    5
8850157527989    5
8850157400190    5
8996001350584    5
                ..
8906159410245    2
8858851439324    2
8935005800008    2
8936076452042    2
8854575001502    2
Name: count, Length: 2060, dtype: int64

In [13]:
df_grouped = df.groupby(['barcode', 'itemName', 'divisionCode', 'divisionName', 'categoryCode', 'categoryName', 'subCategoryCode', 'subCategoryName']).agg({
    'itemQty': 'sum',
    'grossSales': 'sum',
    'itemDiscount': 'sum',
    'netSales': 'sum',
    'totalCostPrice': 'sum'
}).reset_index()

df_grouped

,barcode,itemName,divisionCode,divisionName,categoryCode,categoryName,subCategoryCode,subCategoryName,itemQty,grossSales,itemDiscount,netSales,totalCostPrice
0,21202010,HH Cam Úc 1 trái,29,Fresh Grocery,2901,Fruit Cat,290101,Fruit Sub,7,280000,0,266665,192500.0
1,23133289,GPS Thạch hồng sâm Innerset 15g x gói,7,Beverage,708,Others Beverage,70801,Condition/Supplements,48,1392000,0,1288891,850910.4
2,45116568,Lotte Kem mochi Yukimi Daifuku 90ml x hộp,5,Ice Cream,501,Packaged Ice Cream/Novelties,50104,Other Ice Cream,151,5023000,0,4650961,3294518.0
3,50357680,FISHERMANS Kẹo cay con tau khuynh diep 25g x1 goi,9,Candy,901,Candy Category,90101,Hard Candy,3,66000,11000,61111,50490.0
4,50357680,Fishermans Kẹo Cay Con Tau Khuynh Diep 25g x Gói,9,Candy,901,Candy Category,90101,Hard Candy,6,142000,4000,131480,106029.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3752,9556247536877,Beryl Almond Coated With Dark Chocolate 45g x ...,9,Candy,902,Chocolate,90202,Chocolate Pack,4,152000,0,140740,90240.0
3753,9556247536877,Beryl Almond Coated With Dark Chocolate 45g x Gói,9,Candy,902,Chocolate,90202,Chocolate Pack,3,114000,0,105555,73440.0
3754,18935012413328,THÙNG 24SABECO Lager Bia 330ml x Lon,6,Beer,602,Domestic Beer,60201,Domestic Beer Can,4,1280000,0,1163636,923344.0
3755,28935049502337,THÙNG 24_COCA Zero Sleek 320ml x 1 Lon,7,Beverage,702,Carbonated Drinks - CSD,70202,No/Less Sugar CSD,1,194000,66000,179630,199560.0


In [17]:
top_20_barcodes = df_grouped.sort_values('netSales', ascending=False).head(20)
top_20_barcodes

,barcode,itemName,divisionCode,divisionName,categoryCode,categoryName,subCategoryCode,subCategoryName,itemQty,grossSales,itemDiscount,netSales,totalCostPrice
97,260202000017,Tteokbokki Signature x tô,26,FF Onsite,2602,Other Cuisine,260202,Korean cuisine Sub,476,14280000,0,13222326,8509789.92
3331,8938502525368,Bánh bao nhân thịt heo trứng muối xá xíu,26,FF Onsite,2604,Steamed Bun,260402,Savoury Buns,404,11716000,0,10848203,7378959.00
248,2250103000058,TP. Bánh giò nhân thịt 150g x 1 cái,26,FF Onsite,2604,Steamed Bun,260403,Others Steamed bun,754,10556000,0,9774102,5792745.24
204,2010805000363,Hotdog 25 Signature x cái,26,FF Onsite,2602,Other Cuisine,260201,Other Cuisine Sub,374,10456000,16000,9681510,7125885.58
374,2701010000179,GCCT Cơm nghêu mẹ nấu x hộp,27,FF Offsite,2701,Rice Offsite,270101,Bento,274,10412000,0,9640690,6302000.00
96,260202000017,Tteokbokki Signature x 1 phần,26,FF Onsite,2602,Other Cuisine,260202,Korean cuisine Sub,329,9870000,0,9138962,6065447.44
3328,8938502525337,BB 1 trứng cút 120g x 1 Cái,26,FF Onsite,2604,Steamed Bun,260402,Savoury Buns,518,8772000,34000,8122310,4753271.60
3329,8938502525344,BB 2 trứng cút 150g x 1 Cái,26,FF Onsite,2604,Steamed Bun,260402,Savoury Buns,455,8645000,0,8004778,5556378.10
2630,8935335400466,Bánh bao xá xíu phô mai 120g x 1 cái,26,FF Onsite,2604,Steamed Bun,260402,Savoury Buns,334,8350000,0,7731434,5045988.50
298,2501031190510,Trà tắc ly lớn x 1 ly,25,FF Beverage,2506,FF Beverage Onsite,250603,Cold Local Drinks,457,8230000,910000,7620471,4642891.50


In [25]:
# Filter item Milo 2010103000010
df[df['barcode'] == 2602020019048]

,barcode,itemName,divisionCode,divisionName,categoryCode,categoryName,subCategoryCode,subCategoryName,itemQty,grossSales,itemDiscount,netSales,totalCostPrice
1746,2602020019048,Tteokbokki Signature (TC) x tô,26,FF Onsite,2602,Other Cuisine,260202,Korean cuisine Sub,103,3090000,0,2861134,1966167.0


In [22]:
df_summed = df.groupby('barcode').agg({
    'itemQty': 'sum',
    'grossSales': 'sum',
    'itemDiscount': 'sum',
    'netSales': 'sum',
    'totalCostPrice': 'sum'
}).reset_index()

df_summed

,barcode,itemQty,grossSales,itemDiscount,netSales,totalCostPrice
0,21202010,7,280000,0,266665,192500.0
1,23133289,48,1392000,0,1288891,850910.4
2,45116568,151,5023000,0,4650961,3294518.0
3,50357680,9,208000,15000,192591,156519.0
4,50854011,34,790000,65000,731476,590733.0
...,...,...,...,...,...,...
2448,9556247504111,4,260000,0,240740,157440.0
2449,9556247536877,8,304000,0,281480,186240.0
2450,18935012413328,4,1280000,0,1163636,923344.0
2451,28935049502337,1,194000,66000,179630,199560.0


In [27]:
df_itemName = df.groupby('barcode')['itemName'].first()
df_summed['itemName'] = df_summed['barcode'].map(df_itemName)
df_summed

,barcode,itemQty,grossSales,itemDiscount,netSales,totalCostPrice,itemName
0,21202010,7,280000,0,266665,192500.0,HH Cam Úc 1 trái
1,23133289,48,1392000,0,1288891,850910.4,GPS Thạch hồng sâm Innerset 15g x gói
2,45116568,151,5023000,0,4650961,3294518.0,Lotte Kem mochi Yukimi Daifuku 90ml x hộp
3,50357680,9,208000,15000,192591,156519.0,FISHERMANS Kẹo cay con tau khuynh diep 25g x1 goi
4,50854011,34,790000,65000,731476,590733.0,FISHERMANS Kẹo cay con tàu vị bạc hà 25g x1 Gói
...,...,...,...,...,...,...,...
2448,9556247504111,4,260000,0,240740,157440.0,# Beryl Classic Tiramisu Almond White Chocolat...
2449,9556247536877,8,304000,0,281480,186240.0,Beryl Almond Coated With Dark Chocolate 45g x ...
2450,18935012413328,4,1280000,0,1163636,923344.0,THÙNG 24SABECO Lager Bia 330ml x Lon
2451,28935049502337,1,194000,66000,179630,199560.0,THÙNG 24_COCA Zero Sleek 320ml x 1 Lon


In [28]:

top_20_barcodes = df_summed.sort_values('netSales', ascending=False).head(20)
top_20_barcodes

,barcode,itemQty,grossSales,itemDiscount,netSales,totalCostPrice,itemName
61,260202000017,805,24150000,0,22361288,14575237.36,Tteokbokki Signature x 1 phần
124,2010103000010,595,14830000,45000,13731393,8719692.16,Milo ly lớn x 1ly
139,2010805000363,442,12360000,16000,11444478,8495448.37,Hotdog 25 Signature
2188,8938502525368,404,11716000,0,10848203,7378959.00,Bánh bao nhân thịt heo trứng muối xá xíu
245,2701010000179,299,11362000,0,10520315,6877000.00,GCCT.Cơm nghêu mẹ nấu x 1 hộp
162,2250103000058,754,10556000,0,9774102,5792745.24,TP. Bánh giò nhân thịt 150g x 1 cái
189,2501031190510,561,10310000,910000,9546441,5700664.50,Trà tắc ly lớn x 1 ly
177,2270103000100,267,10146000,0,9394395,5958497.00,Gimbap dakgalbi phô mai
2179,8938502004030,390,9750000,0,9027720,4875000.00,HTH Trà sữa olong lài 250ml x chai
187,2501031190374,840,9740000,340000,9018434,4502544.00,Nestea ly lớn (New) x 1 ly
